In [ ]:
import os
from s3fs.core import S3FileSystem
from xml.etree import ElementTree as ET
import pandas as pd


os.environ['AWS_CONFIG_FILE'] = 'aws_config.ini'

fs = S3FileSystem(anon=False)
bucket = 'treetracker-training-images'
datadir = os.path.join(bucket, "PlantCLEF_2016")


In [7]:

def parse_xml_s3path(s3path):
    '''
    Parse and return a PlantCLEF XML file given a valid s3path
    '''
    if os.path.splitext(s3path)[1] == ".xml": 
        with fs.open(s3path) as file:
            tree = ET.parse(file)
            species = tree.find("Species").text
            family = tree.find("Family").text
            genus = tree.find("Genus").text
            class_id = tree.find("ClassId").text
            obs_id = tree.find("ObservationId").text
            date = tree.find("Date").text
            role = tree.find("LearnTag").text
            latitude = tree.find("Latitude").text
            longitude = tree.find("Longitude").text
            return (role, obs_id, class_id, date, family, genus, species, latitude, longitude)
    else:
        pass

def generate_plantclef_csv(write_csv, batch_size=100):
    '''
    Collects metadata from PlantCLEF 2016 and creates csv file.
    Originally written to s3://treetracker-training-images/PlantCLEF_2016/plantclef.csv.
    '''
    linkings_dict = {}
    if fs.exists(write_csv):
        already_done = dict.fromkeys(pd.read_csv(write_csv).index.values)
        count = len(already_done)
        print (count, " from earlier run already finished")
    else:
        already_done = []
        count = 0
        print ("Starting from scratch")
        time.sleep(5)
    for f, e, d in fs.walk(datadir):
        for path in d:
            if os.path.splitext(path)[1] == ".jpg": # Image file
                base = os.path.splitext(path)[0]
                fullpath = os.path.join(f, base)
                xmlpath = fullpath + ".xml"
                count += 1
                if count % batch_size == 0:
                    linkings = pd.DataFrame.from_dict(linkings_dict, orient="index")
                    if count <= batch_size and len(already_done) == 0: # first run ever
                        print ("Creating csv")
                        linkings.columns = ["role", "obs_id", "class_id", "date", "family", "genus", "species", "latitude", "longitude"]
                        linkings.to_csv(write_csv, mode='w', index=False)
                    else:
                        print ("%d done appending to %s"%(count, write_csv))
                        linkings.to_csv(write_csv, mode='a', header=False, index=False) # append batch-wise
                    linkings_dict.clear()
                if not fs.exists(xmlpath):
                    print ("File ", xmlpath , " does not exist!")
                if already_done.get(base, 0) == 0: # makes sure we're not processing something that already has been processed
                    print (base)
                    linkings_dict [base] = parse_xml_s3path(xmlpath)
                
    return pd.read_csv(write_csv)

            
            
    
    

In [38]:
plantclef_data = pd.read_csv("s3://" + datadir + "/plantclef.csv")


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
display (plantclef_data.drop_duplicates())

,filename,role,obs_id,class_id,date,family,genus,species,latitude,longitude
0,1,Train,21782,5810,2013-3-21,Amaryllidaceae,Narcissus,Narcissus dubius Gouan,42.98235,3.01486
1,10,Train,11786,6448,2014-8-22,Orchidaceae,Epipactis,Epipactis atrorubens (Hoffm.) Besser,45.34137,5.81565
2,1000,Train,40480,30040,2011-4-8,Fabaceae,Cercis,Cercis siliquastrum L.,NaN,NaN
3,10000,Train,14235,3529,2013-9-12,Lamiaceae,Clinopodium,Clinopodium nepeta (L.) Kuntze,NaN,NaN
4,100001,Train,33815,4736,2013-6-5,Rosaceae,Cydonia,Cydonia oblonga Mill.,48.84059,2.36158
...,...,...,...,...,...,...,...,...,...,...
132285,20812,Train,10548,5324,2012-11-26,Plantaginaceae,Digitalis,Digitalis purpurea L.,-21.1053,55.38092
132287,20814,Train,119,5537,2013-6-6,Simaroubaceae,Ailanthus,Ailanthus altissima (Mill.) Swingle,43.598,3.89899
132290,20817,Train,25374,30136,2006-3-20,Amaryllidaceae,Galanthus,Galanthus nivalis L.,47.3466,-1.29023
132297,20825,Train,14938,3806,2012-6-26,Malvaceae,Althaea,Althaea officinalis L.,48.0289,0.18775
